In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import copy

from PIL import Image
from sklearn.metrics import accuracy_score

## Download dataset 
https://drive.google.com/drive/folders/1vI8Bkk5DojitLNkpz-UT30jOEay0XXon?usp=sharing

## Load data

In [8]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")
y_train = y_train.reshape(len(y_train))

x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")
y_test = y_test.reshape(len(y_test))

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [10]:
# It's a multi-class classification problem 
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6,'horse': 7,'ship': 8, 'truck': 9}
print(np.unique(y_train))

[0 1 2 3 4 5 6 7 8 9]


![image](https://img-blog.csdnimg.cn/20190623084800880.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lqcDE5ODcxMDEz,size_16,color_FFFFFF,t_70)

## Data preprocess

In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to one-hot encoding (keras model requires one-hot label as inputs)
num_classes = 10
print(y_train[0])
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[0])

[9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


## Build model & training (Keras)

In [9]:
# Builde model
model = Sequential() # Sequential groups a linear stack of layers 
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=x_train.shape[1:])) # Add Convolution layers
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(Conv2D(filters=32, kernel_size=(3, 3))) # Add Convolution layers
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(MaxPooling2D(pool_size=(4, 4))) # Add Max pooling to lower the sptail dimension

model.add(Flatten()) # Flatten the featuremaps
model.add(Dense(units=512)) # Add dense layer with 512 neurons
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(Dense(units=num_classes)) # Add final output layer for 10 classes
model.add(Activation('softmax')) # Add softmax activation to transfer logits into probabilities

# initiate SGD optimizer
opt = keras.optimizers.SGD()

# Compile the model with loss function and optimizer, and evaluate with accuracy
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Setup some hyperparameters
batch_size = 32
epochs = 10

# Fit the data into model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 24s 478us/step - loss: 1.8933 - acc: 0.3203 - val_loss: 1.5972 - val_acc: 0.4367
Epoch 2/10
50000/50000 [==============================] - 43s 867us/step - loss: 1.4593 - acc: 0.4818 - val_loss: 1.3575 - val_acc: 0.5240
Epoch 3/10
50000/50000 [==============================] - 34s 686us/step - loss: 1.2797 - acc: 0.5500 - val_loss: 1.2541 - val_acc: 0.5644
Epoch 4/10
50000/50000 [==============================] - 36s 720us/step - loss: 1.1661 - acc: 0.5923 - val_loss: 1.2103 - val_acc: 0.5730
Epoch 5/10
50000/50000 [==============================] - 37s 744us/step - loss: 1.0775 - acc: 0.6257 - val_loss: 1.0946 - val_acc: 0.6187
Epoch 6/10
50000/50000 [==============================] - 45s 908us/step - loss: 1.0061 - acc: 0.6518 - val_loss: 1.0458 - val_acc: 0.6334
Epoch 7/10
50000/50000 [==============================] - 29s 589us/step - loss: 0.9396 - acc: 0.6733 - val_loss: 0.

In [10]:
y_pred = model.predict(x_test)
print(y_pred.shape) # 10000 samples, each sample with probaility of 10 classes

(10000, 10)


In [11]:
y_pred[0] 

array([2.4284909e-03, 2.9643339e-01, 1.2415329e-03, 3.2413865e-03,
       1.1167271e-03, 1.0127937e-03, 6.8496183e-06, 3.5009726e-03,
       3.1431669e-03, 6.8787467e-01], dtype=float32)

In [12]:
np.argmax(y_pred[0]) # argmax to find the predict class with highest probability. 9=truck

9

In [13]:
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test-set:  0.6769


## My Model

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

img = Image.fromarray(x_train[100])
tmp = data_transforms['train'](img)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __getitem__(self, index):
        X = self.X[index]

        if self.transform:
            X = self.transform(Image.fromarray(X))

        y = self.y[index]

        return X, y

    def __len__(self):
        return self.y.shape[0]

In [ ]:
# split dataset
datasets = {}
train_size = 47000

datasets['train'] = CustomDataset(x_train[:train_size], y_train[:train_size], data_transforms['train'])
datasets['val'] = CustomDataset(x_train[train_size:], y_train[train_size:], data_transforms['val'])
datasets['test'] = CustomDataset(x_test, y_test, data_transforms['test'])

In [ ]:
dataloaders = {x: DataLoader(datasets[x], batch_size=128, shuffle=True)
              for x in ['train', 'val'] }
dataloaders['test'] = DataLoader(datasets['test'], batch_size=2000, shuffle=False)
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'val', 'test']}
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
print(dataset_sizes['train'])
print(dataset_sizes['val'])
print(dataset_sizes['test'])

### Pretrained Model

In [ ]:
model_conv = torchvision.models.resnet152(pretrained=True)
# for param in model_conv.parameters():
#     param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 10)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
# optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.01, momentum=0.9, weight_decay=?)
optimizer_conv = optim.Adam(model_conv.parameters(), 
                                  lr=0.003, 
                                  betas=(0.9, 0.999), 
                                  eps=1e-08, 
                                  weight_decay=1e-3, 
                                  amsgrad=False)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)  

In [ ]:
def train(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        # if epoch > 1:
        #     for param in model.parameters():
        #         param.requires_grad = True

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i, data in enumerate(dataloaders[phase]):
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # for param in model_conv.parameters():
                    #     print(param.requires_grad)
                      
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def predict(model):
    model.eval()
    y_pred = []

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            y_pred.extend(preds.tolist())
    return np.array(y_pred)

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [ ]:
y_pred = predict(model)

In [14]:
assert y_pred.shape == (10000,)

In [ ]:
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))